# Setup

In [10]:
from numpy import random
import itertools
import numpy
import six
from collections import namedtuple, Mapping

# Ising

## Generate graph

In [2]:
NUM_SPINS = 26
NUM_STATES = 10
OMP_THREADS = 4

In [3]:
# Generate graph
graph = {}
for i in range(NUM_SPINS):
    graph[(i, i)] = random.rand() * 10 - 5
    for j in range(i, NUM_SPINS):
        graph[(i, j)] = random.rand() * 10 - 5

In [4]:
graph

{(0, 0): -3.49704922416882,
 (0, 1): -2.5736506339247613,
 (0, 2): 4.589607179922046,
 (0, 3): -0.19437191934234122,
 (0, 4): -3.1469851868797827,
 (0, 5): 2.4955987667675084,
 (0, 6): -2.3307203261039966,
 (0, 7): -4.172824282251753,
 (0, 8): -4.598519083323537,
 (0, 9): -1.5610303482764598,
 (0, 10): -0.6656367850833877,
 (0, 11): -2.713758265882593,
 (0, 12): -4.399574352300401,
 (0, 13): -4.336719112282528,
 (0, 14): 3.932654146674663,
 (0, 15): -2.9716536588091147,
 (0, 16): 1.4893873485629001,
 (0, 17): -0.024365514154461643,
 (0, 18): 3.1894923012135283,
 (0, 19): -2.5284943419500525,
 (0, 20): -4.425197658531028,
 (0, 21): 4.6623990804904025,
 (0, 22): 2.8298526855092563,
 (0, 23): 3.883122447273754,
 (0, 24): -4.985971510897067,
 (0, 25): 3.640664579505458,
 (1, 1): -4.107352468416936,
 (1, 2): -0.7923660542173607,
 (1, 3): -4.543873202175357,
 (1, 4): 4.874440392077615,
 (1, 5): 2.7957000313747056,
 (1, 6): 0.69101588221256,
 (1, 7): 1.3308454182854144,
 (1, 8): -0.2733411800

## Ising problem

In [7]:
IsingProblem = namedtuple('IsingProblem', ['matrix', 'spin_labels'])

In [11]:
# search
omp_threads = 4
num_states=10
chunk_size=20
energies_only=False

In [13]:
jh_mat, labels = read_graph(graph)

qubo, const = ising_to_qubo(jh_mat)


# Utils

In [12]:
def read_graph(graph):
    """Read Hamiltonian and spin labels from given graph.

    :param graph: a definition of Ising problem in one of the following formats:
     - as a Mapping of spin indices (i, j) to the corresponding coupler (or
       spin weight if i == j).
     - as a row-format, i.e. an iterable of (i, j, value) with analogous meaning
       as in Mapping format.
     - as a square matrix (numpy.ndarray or nested lists) such that graph[i, j] corresponds
       to coupling between spins i and j (or a spin weight if i == j).
    :returns: a named tuplew with matrix and spin_labels components. The matrix
     is a square upper triangular 2D numpy.ndarray, upper triangular, encoding
     system Hamiltonian (without lower triangle unneeded for computations).
     The spin_labels is a tuple of integers mapping spins of system represented
     by the matrix into spins into original graph.
    :rtype: IsingProblem
    """

    if isinstance(graph, Mapping):
        return _ising_from_mapping(graph)
    detected_shape = numpy.ma.shape(graph)
    if len(detected_shape) != 2:
        raise ValueError('Unsupported graph format specified. Consult docs.')
    if detected_shape[1] == 3 and has_possibly_spin_labels(graph):
        return _ising_from_rows(graph)

    if detected_shape[0] == detected_shape[1]:
        return _ising_from_matrix(graph)

    raise ValueError('Unsupported graph format specified. Consult docs.')

def _ising_from_matrix(graph):
    graph = numpy.array(graph)
    jh_matrix = numpy.zeros(graph.shape)
    for i in range(graph.shape[0]):
        for j in range(graph.shape[1]):
            first, second = sorted((i, j))
            jh_matrix[first, second] += graph[i, j]
    return IsingProblem(jh_matrix, tuple(range(jh_matrix.shape[0])))

def _ising_from_mapping(graph_map):
    seen_spins = set()
    for i, j in graph_map:
        seen_spins.add(i)
        seen_spins.add(j)
    system_size = len(seen_spins)
    jh_matrix = numpy.zeros(shape=(system_size, system_size))
    spin_labels = tuple(sorted(seen_spins))
    spin_map = {spin: idx for idx, spin in enumerate(spin_labels)}
    for (i, j), value in six.iteritems(graph_map):
        first, second = sorted((spin_map[i], spin_map[j]))
        jh_matrix[first, second] += value
    return IsingProblem(jh_matrix, spin_labels)

def ising_to_qubo(ham):
    qubo = numpy.zeros(ham.shape)
    constant = 0
    for i in range(qubo.shape[0]):
        qubo[i, i] = 2 * ham[i, i]
        constant +=  ham[i, i]
        for j in range(qubo.shape[0]):
            if i != j:
                low, high = sorted((i, j))
                constant -= ham[low, high] * 0.5
                qubo[i, i] -= 2 * ham[low, high]
                qubo[low, high] += ham[low, high] * 2
    return qubo, constant

In [82]:
def spin_variable_generator(n):
    for k, s in enumerate(itertools.product([-1,1],repeat=n)):
        yield s

def qubo_variable_generator(n):
    for k, s in enumerate(itertools.product([0,1],repeat=n)):
        yield s

def bits_to_int(bits):
    bits = np.asarray(bits)
    
    return np.right_shift(np.packbits(bits, -1), bits.size)

In [91]:
def dummy_solve_qubo(qubo, n, num_states=10):
    res = {}
    
    for k, q in enumerate(qubo_variable_generator(n)):
        F = 0
        for i in range(qubo.shape[0]):
            F -= qubo[i,i]*q[i]
            for j in range(qubo.shape[0]):
                low, high = sorted([i, j])
                F -= qubo[low, high] * q[i] * q[j]
        b = bits_to_int(q)[0]
        res[b] = F
        
    res = dict(sorted(res.items(), key=lambda item: item[1]))
    
    if num_states > len(res):
        return res
    else:
        return dict(itertools.islice(res.items(), num_states)) 

In [70]:
NS = 4

graph = {}
for i in range(NS):
    graph[(i, i)] = random.rand() * 10 - 5
    for j in range(i, NS):
        graph[(i, j)] = random.rand() * 10 - 5

In [71]:
jh_mat, labels = read_graph(graph)

qubo, const = ising_to_qubo(jh_mat)

In [92]:
dummy_solve_qubo(qubo, NS)

{11: -32.392125660862604,
 4: -19.174932341287622,
 12: -3.1209233211956047,
 0: 0.0,
 15: 4.696249235545961,
 10: 6.300871609285053,
 3: 8.49127574284163,
 14: 10.487300500471523,
 9: 14.814001670031296,
 2: 17.749034945772003}

# TMP

In [28]:
def search(graph, num_states=10, energies_only=False, **kwargs):
    """Find lowest energies and corresponding states of Ising Model specified by Jh.

    :param Jh: matrix encoding hamiltonian of the sytem. Should be real and symmetric.
    :type Jh: numpy.ndarray or array-like
    :param sweep_size: number of bits being changed during a single sweep during exhaustive
     search. Has to obey `1 <= sweep_size <= Jh.shape[0]`.
    :type sweep_size: int
    :param num_states: how many lowest states to keep. Default is 10.
    :type num_states:
    :returns: namedtuple with components `states` and `energies` holding computed states of
     lowest energy and corresponding energies.
    :rtype: :py:class:`EnergiesAndStates
    """
    logger = logging.getLogger('ising')

    jh_mat, labels = read_graph(graph)

    qubo, const = ising_to_qubo(jh_mat)
    method = choose_method(gpusearch, **kwargs)
    logger.info('Choosen computation mehtod: %s', method)

    if 'chunk_size' in kwargs:
        chunk_size = kwargs['chunk_size']
        logger.debug('Chunk size given explicitly as %f.', chunk_size)
    else:
        logger.debug('Deducing chunk size from the available memory.')
        chunk_size = max_chunk_size_for_method(method)

    if chunk_size > jh_mat.shape[0]:
        logger.debug('Clipping chunk size to the number of spins.')
        chunk_size = jh_mat.shape[0]

    logger.info('Chunk size exponent that will be used: %d', chunk_size)

    if num_states > 2 * 2 ** chunk_size:
        logger.warning('Requested more states than two chunks. Clipping at 2 ** (chunk size)')
        num_states = 2 ** chunk_size

    if kwargs.get('show_progress', False):
        pbar = progressbar.ProgressBar(max_value=2 ** (jh_mat.shape[0]-chunk_size))
        callback = pbar.update
    else:
        pbar = None
        callback = dummy_callback

    if method.lower() == 'cpu':
        logger.info('Running CPU search: Jh.shape=%s, chunk_size=%d, num_states=%d',
                    jh_mat.shape, chunk_size, num_states)
        callback(0)
        if energies_only:
            energies = cpusearch.find_lowest_energies_only(qubo,
                                                           chunk_size,
                                                           num_states,
                                                           callback=callback)
        else:
            energies, state_reprs = cpusearch.find_lowest(qubo,
                                                          chunk_size,
                                                          num_states,
                                                          callback=callback)
    else:
        logger.info('Running GPU search: Jh.shape=%s, num_states=%d, chunk_size=%d',
                    jh_mat.shape, num_states, chunk_size)
        callback(0)
        if energies_only:
            energies = gpusearch.find_lowest_energies_only(qubo,
                                                           chunk_size,
                                                           num_states,
                                                           callback=callback)
        else:
            energies, state_reprs = gpusearch.find_lowest(qubo,
                                                          chunk_size,
                                                          num_states,
                                                          callback=callback)
    if pbar is not None:
        pbar.finish()
    if energies_only:
        state_reprs = None
    return EnergiesAndStates(raw_states=state_reprs, labels=labels, energies=energies + const)